In [1]:
import os

In [2]:
%pwd

'd:\\recommendation-engine\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\recommendation-engine'

In [5]:
import pandas as pd

In [6]:
data1 = pd.read_csv("artifacts/data_ingestion/generated_data/bookings.csv")
data2 = pd.read_csv("artifacts/data_ingestion/generated_data/comments.csv")
data3 = pd.read_csv("artifacts/data_ingestion/generated_data/likes.csv")
data1.head()
data2.head()
data3.head()

,user_id,liked_event_id,liked_event_organizer_id,like_id
0,dd273811-54c2-4c13-b0a5-fca4f3b621b8,66fa3a5d-fb3d-4faf-a295-2b7456f18620,1aafbaa0-9efd-4d51-adbb-310e8a3355d5,12f33ebf-2424-43b6-a72d-d8bcdc9ce806
1,09c23f75-ed99-49e7-8a34-2d9eb0c7f991,87ab34fa-e294-4f40-a51a-546079da2463,53da9a59-b5be-4ce2-bd73-e79826611b1c,e6e31bc3-219d-49e2-8344-3bb30b9e5fcc
2,f81fffe9-8813-48f9-87dc-01853f46e603,6b660a1b-07da-4c64-876e-03542029ed15,fcd1270d-0dd3-41ee-b999-7d39768c3369,cb4e26e2-76fa-4042-bb7f-2433dcf4ca2f
3,bd4d04b9-98cc-4937-bf84-f3e092a30758,39def82b-1f0a-4278-9d37-1ce25daf4cae,d86d4720-3bf8-49f9-85e3-a2d1c37d4cce,97e6060e-83ed-4510-9dcd-01b16614b867
4,cec3a311-1115-4f88-bf7b-8f2e67b59196,ba8c9541-9d61-4163-bc64-d613685ef041,85ddc80b-ecf1-4895-b690-d1d3f23c9576,7cea1a78-d43b-4120-bab5-159fc832001b


In [7]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 4 columns):
 #   Column                     Non-Null Count   Dtype 
---  ------                     --------------   ----- 
 0   user_id                    150000 non-null  object
 1   booked_event_id            150000 non-null  object
 2   booked_event_organizer_id  150000 non-null  object
 3   booking_id                 150000 non-null  object
dtypes: object(4)
memory usage: 4.6+ MB


In [8]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    merged_data_url: Path
    unzip_data_dir: Path
    all_schema: dict

In [9]:
from src.hybrid_recommender.constants import *
from src.hybrid_recommender.utils.common import read_yaml, create_directories

In [10]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            unzip_data_dir = config.unzip_data_dir,
            merged_data_url = config.merged_data_url,
            all_schema=schema,
        )

        return data_validation_config

In [11]:
import os
from src.hybrid_recommender import logger


In [12]:
class DataValiadtion:
    def __init__(self, config: DataValidationConfig):
        self.config = config


    def validate_all_columns(self)-> bool:
        try:
            validation_status = None

            data = pd.read_csv(self.config.merged_data_url)
            all_cols = list(data.columns)

            all_schema = self.config.all_schema.keys()

            
            for col in all_cols:
                if col not in all_schema:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")

            return validation_status
        
        except Exception as e:
            raise e

In [13]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValiadtion(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e

[2025-07-01 12:12:35,496: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-07-01 12:12:35,496: INFO: common: yaml file: params.yaml loaded successfully]
[2025-07-01 12:12:35,609: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-07-01 12:12:35,609: INFO: common: created directory at: artifacts]
[2025-07-01 12:12:35,609: INFO: common: created directory at: artifacts/data_validation]
